# Library import

In [2]:
import os
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import wandb

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device = {device}")
torch.cuda.empty_cache()

device = cpu


# Hyperparameters

In [5]:
model_name = "Clean-5-Convnet"
eval_every = 1
save_every = 5

# est-ce qu'on pourrait pas faire des batch size différent pour le train et le test/validation ?
# as-tu d'autres idée d'hyperparametre ?
batch_size = 128
learning_rate = 0.001
nb_epochs = 50
proba_dropout = 0.2
conv_kernel_size = 3
pooling_kernel_size = 2

In [14]:
# if ask for a key, COPY/PASTE bc0dc5f66039bf86ffba73188a299faed907973c
wandb.init(
    project="deep_learning_project", 
    config = {
        "model": model_name,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "nb_epochs": nb_epochs,
        "eval_every": eval_every,
        "save_every": save_every,
        "conv_kernel_size": conv_kernel_size,
        "pooling_kernel_size": pooling_kernel_size,
        "proba_dropout": proba_dropout,
        "kernel_incr": "double",
        "Padding": "True",
        "BatchNorm": "True",
    },
    name=model_name,
    tags = ["test"],
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/florenthervers/.netrc


# Dataset import

In [6]:
import datasetCreation

root_dir = "/scratch/users/jwalraff/DeepWalraff/augmented_dataset"
#root_dir = "../augmented_dataset"
test_dir = os.path.join(root_dir, "test")
categories = os.listdir(test_dir)

transform = transforms.Compose([
    transforms.ToTensor()
])

train_loader = datasetCreation.getDataLoader(root_dir, batch_size, num_workers= 4, mode='train', transform=transform)
validation_loader = datasetCreation.getDataLoader(root_dir, batch_size, num_workers= 4, mode='validation', transform=transform)
test_loader = datasetCreation.getDataLoader(root_dir, batch_size, mode='test', transform=transform)

# Class Weights

In [7]:
# source of the formula https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/
weights = []
category_count = []
n_category = len(categories)

for i, cat in enumerate(categories):
    tmp = os.path.join(test_dir, cat)
    category_count.append(len(os.listdir(tmp)))

n_sample = sum(category_count)    
    
for i, n_sample_i in enumerate(category_count):
    weights.append(n_sample/(n_category*n_sample_i))

weights = torch.tensor(weights, dtype=torch.float).to(device)
weights

tensor([1.0644, 9.1866, 1.0056, 0.5781, 0.8338, 0.8243, 1.2271])

# Model definition

In [12]:
class ConvNet5Layers(nn.Module):
    def __init__(self, kernel_size, proba_dropout, pooling_kernel_size):
        # basic architecture from slide 35/59 lecture 6 of deep learning course
        # INPUT → [[CONV → ReLU]*N → POOL?]*M → [FC → ReLU]*K → FC
        super(ConvNet5Layers, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=kernel_size, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=pooling_kernel_size)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=kernel_size, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=pooling_kernel_size)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=kernel_size, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=pooling_kernel_size)
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=kernel_size, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=pooling_kernel_size)
        
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=kernel_size, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(kernel_size= pooling_kernel_size)
        
        self.fc1 = nn.Linear(3584, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.dropout6 = nn.Dropout(proba_dropout)
        
        self.fc2 = nn.Linear(512, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout7 = nn.Dropout(proba_dropout)
        
        self.fc3 = nn.Linear(512, 7)  # 7 classes en sortie
        
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.pool1(x)
        
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool2(x)
        
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.pool3(x)
        
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.pool4(x)
        
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.pool5(x)
        
        x = torch.flatten(x, 1)
        
        x = self.bn6(F.relu(self.fc1(x)))
        x = self.dropout6(x)
        
        x = self.bn7(F.relu(self.fc2(x)))
        x = self.dropout7(x)
        
        x = self.fc3(x)
        return x

In [13]:
convnet = ConvNet5Layers(conv_kernel_size, proba_dropout, pooling_kernel_size)
convnet.train()

print(f"Model architecture: \n{convnet}\n")

# from https://saturncloud.io/blog/check-the-total-number-of-parameters-in-a-pytorch-model/#:~:text=To%20check%20the%20number%20of%20parameters%20in%20a%20PyTorch%20model,its%20total%20number%20of%20elements
print(f"Numbers of parameters: {sum(p.numel() for p in convnet.parameters())}")

# Loss and optimizer
# Je rajoute les poids ici pour pas faire ton calcul chelou pour la loss, c'est fait pour.
criterion = nn.CrossEntropyLoss(weight=weights, reduction='none')
optimizer = torch.optim.Adam(convnet.parameters(), lr=learning_rate)

Model architecture: 
ConvNet5Layers(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, s

# Training loop

In [16]:
def train_cnn(num_epochs):
    # transfer cnn to GPU (in-place)
    convnet.to(device)

    print("Entering epoch loops.....")
    for i in range(num_epochs):
        print("Entering train loop....")
        # variables to keep results across the batches
        train_losses = []
        validation_losses = []

        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            pred = convnet(x)
            loss = criterion(pred, y)

            train_losses.append(loss.cpu().detach())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        wandb.log({
                "epoch": i, 
                "train_loss": np.array(train_losses).mean()
            })

        # à modifier en fonction du nb d'epoch qu'on met
        if i % eval_every == 0:
            print("Entering validation.....")
            with torch.no_grad():
                #disable the dropout during the evaluation
                convnet.eval()

                # variables used for the metrics
                validation_true_labels = torch.tensor([])
                validation_pred_labels = torch.tensor([])
                validation_predicted_probas = torch.tensor([])
                class_correctly_predicted = np.zeros(7)
                class_totals = np.zeros(7)
                
                for x, y in validation_loader:
                    x = x.to(device)
                    y = y.to(device)

                    pred = convnet(x)
                    # convert the output in probabilities
                    probabilities = F.softmax(pred, dim=1)
                    
                    # get the predictions with their associated probabilities
                    batch_predicted_proba, batch_predicted_classes_indexes = torch.max(probabilities, dim=1)
                    
                    # update the per-classes accuracy stats for the batch
                    for batch_index in range(len(batch_predicted_classes_indexes)):
                        # fetch the true and predicted classes
                        predicted_class = batch_predicted_classes_indexes[batch_index]
                        true_class = y[batch_index]
                        
                        # save the results to compute metrics on the whole validation set
                        validation_true_labels = torch.cat((validation_true_labels, true_class.cpu()))
                        validation_pred_labels = torch.cat((validation_pred_labels, predicted_class.cpu()))
                        
                        # update the per-class metrics variables
                        class_totals[true_class] += 1
                        if predicted_class == true_class:
                            class_correctly_predicted[true_class] += 1
                    
                    # save the predicted probas and losses
                    validation_predicted_probas = torch.cat((validation_predicted_probas, batch_predicted_proba.cpu()))
                    validation_losses.append(criterion(pred, y).cpu().detach())

                # compute and log the global metrics on the whole validation scale
                wandb.log({
                    "epoch": i,
                    "confidence": np.array(validation_predicted_probas).mean(),
                    "accuracy":accuracy_score(validation_true_labels, validation_pred_labels),
                    "validation_loss":  np.array(validation_losses).mean(),
                    "f1": f1_score(validation_true_labels, validation_pred_labels, average='weighted'),
                    "precision":precision_score(validation_true_labels, validation_pred_labels, average='weighted'),
                    "recall": recall_score(validation_true_labels, validation_pred_labels, average='weighted')
                })
                
                # compute and log the per class metrics
                class_accuracy = [class_correctly_predicted[c] / class_totals[c] for c in range(7)]
                class_accuracy_variance = np.var(np.array(class_accuracy))
                
                for c in range(7):
                    wandb.log({
                        "epoch": i,
                        f"class_accuracy_{c}": class_accuracy[c]
                    })
                    
                wandb.log({
                    "epoch": i,
                    "class_accuracy_variance": class_accuracy_variance
                })
                convnet.test()

        print(f"Epoch {i}: finished")
        print("************************************")

        if i % save_every == 0:
            torch.save(convnet.state_dict(), f"model_intermediate/convnet_{i}.pt")

    torch.save(convnet.state_dict(), f"model_finished/convnet__finished_{num_epochs}.pt")

train_cnn(num_epochs=nb_epochs)

Entering epoch loops.....
Entering train loop....


: 